<hr style="border:2px solid black"></hr>

# Initialization

In [2]:
from toolbox.initialize import *

Creating a Spark session.
	Execution time: 68.82851 s.
Getting the data paths.
	Number of folders: 62
	Number of files: 275621
Loading data from path: "/data/work/src/musicactivity".
	Execution time: 8.36702 s.
Loading data from path: "/data/work/shared/s001284/Music_Project/resources/data/df_sample_raw_1E6.parquet".
	Execution time: 1.47982 s.


<hr style="border:2px solid black"></hr>

# Cleaning

### Choose dataframe

In [3]:
df = df_raw

### Removing rows where activity has been deleted

In [4]:
# Save the start time for timing.
start_time = time.time()

df_clean = (
    df
    .where(f.col('deleted_time').isNull()
           | (f.col('deleted_time') == ''))
)

# Show the execution time.
print(f'Execution time: {time.time() - start_time:.5f} s.')

# Show the top rows of the resulting database.
df_clean.limit(100).toPandas().head(5)

Execution time: 0.17754 s.


,id,deleted_time,useruuid,start_time,end_time,devices,tracks,yearmonth
0,4302e6a4-d970-404f-a7a8-4297164d6aa6-2016-10-16,,92e17ec9-982c-4e02-8ce4-76b19728ddf7,2016-10-16T18:31:56.512+08:00,2016-10-16T18:52:01.608+08:00,"[(E6853, PHONE, e6211e25cf8fd7bc)]","[(2016-10-16T18:31:56.512+08:00, 2016-10-16T18...",201610
1,a6a7c511-239d-4533-b6ae-4f71b1bb6905-2016-10-15,,c705ee6e-703a-41f8-ba0f-0e2b5ef9f4bd,2016-10-15T12:59:27.351+03:30,2016-10-15T13:01:44.724+03:30,"[(D6502, PHONE, 2588c3f87d096e10)]","[(2016-10-15T12:59:27.351+03:30, 2016-10-15T13...",201610
2,cc44fd9c-1333-4f24-b7d6-ed2ffb87a887-2016-10-15,,c705ee6e-703a-41f8-ba0f-0e2b5ef9f4bd,2016-10-15T08:25:21.252+03:30,2016-10-15T08:48:43.198+03:30,"[(D6502, PHONE, 2588c3f87d096e10)]","[(2016-10-15T08:25:21.252+03:30, 2016-10-15T08...",201610
3,e111dd7e-ae55-4609-9b28-f527dfffd541-2016-10-15,,c705ee6e-703a-41f8-ba0f-0e2b5ef9f4bd,2016-10-15T12:49:41.355+03:30,2016-10-15T12:57:41.425+03:30,"[(D6502, PHONE, 2588c3f87d096e10)]","[(2016-10-15T12:49:41.355+03:30, 2016-10-15T12...",201610
4,ce6f2718-f761-41e4-8ad2-f1fb5162f2a3-2016-10-15,,c705ee6e-703a-41f8-ba0f-0e2b5ef9f4bd,2016-10-15T11:00:05.654+03:30,2016-10-15T11:01:12.511+03:30,"[(D6502, PHONE, 2588c3f87d096e10)]","[(2016-10-15T11:00:05.654+03:30, 2016-10-15T11...",201610


### Dropping duplicates of activity id (keeping the most recent)

In [ ]:
# Save the start time for timing
start_time = time.time()

# Define the time format used for the time stamp - strings in the
# raw database.
time_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

# Define a partitioning by activity id.
window_id = Window.partitionBy('id')

# Drop rows containing the duplicates of the activity ID.
# Keep the activity ID with the latest timestamp-
df_clean = (
    df_clean
    
    # Convert the start time to timestamp.
    .withColumn('start_time',
                f.to_timestamp('start_time', time_format)) 
    
    # Save the latest start time for each activity ID in a separate column.
    .withColumn('latest_start_time', f.max('start_time').over(window_id))
    
    # Keep only rows with the latest start time for the given activity ID.
    .where(f.col('start_time') == f.col('latest_start_time'))
        
    # Keep only the first occurrence of each activity ID.
    .dropDuplicates(['id'])
    
    # Delete the column containing the lates start time.
    .drop('latest_start_time')
)

print(f'Execution time: {time.time() - start_time:.5f} s.')

df_clean.limit(100).toPandas().head()

Execution time: 0.11901 s.


### Exploding the columns `devices` and `tracks` and flattening the schema

In [ ]:
# Explore the column devices and tracks
# and flatten the schema
df_clean = t.format_dataframe(df_clean)

# Show the top rows of the resulting dataframe
df_clean.limit(100).toPandas().head(5)

### Replacing start and end time with start time and duration

In [ ]:
# Save the start time for timing
start_time = time.time()

# Define the time format used for the time stamp - strings in the
# raw database.
time_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

df_clean = (
    df_clean
#     .withColumn('start_time',
#                 f.to_timestamp('start_time', time_format))
    .withColumn('end_time',
                f.to_timestamp('end_time', time_format))
    .withColumn('activity_duration',
                f.col('end_time').cast(LongType()) 
                - f.col('start_time').cast(LongType()))   
    .withColumn('tracks_start_time',
                f.to_timestamp('tracks_start_time', time_format))
    .withColumn('tracks_end_time',
                f.to_timestamp('tracks_end_time', time_format))
    .withColumn('track_duration',
                f.col('tracks_end_time').cast(LongType()) 
                - f.col('tracks_start_time').cast(LongType()))
    .drop('end_time')
    .drop('tracks_end_time')
)

# Print the execution time
print(f'Execution time: {time.time() - start_time:.5f} s.')

# Show the resulting dataframe
df_clean.limit(100).toPandas().head(5)

### Merging Track ID and Track URI

I can't decide whether it's a good idea or not. `tracks_id` seems to be a path to a file in the Sony system, which I can't see give any useful information, maybe aside of comparing the track properties of multiple rows with the same track id to look for inconsitencies. I will be dropping the merge for now.

In [ ]:
# df_clean = (
#     df_clean
#     .withColumn('track_id',
#                 f.concat(f.col('tracks_id'), 
#                          f.col('tracks_uri')))
#     .drop('tracks_id')
#     .drop('tracks_uri')
# )

# df_clean.limit(3).toPandas().head()

### Drop redundand columns

Let's remove the columns:

* `deleted_time`
* `devices_type`
* `yearmonth`

In [ ]:
df_clean = (
    df_clean
    .drop('deleted_time')
    .drop('devices_type')
    .drop('yearmonth')
)

df_clean.limit(100).toPandas().head(3)

### Rename and organize columns

In [ ]:
start_time = time.time()

df_clean = (
    df_clean
    
    # Give columns meaningful names.
    .withColumnRenamed('id', 'activity_id')
    .withColumnRenamed('useruuid', 'user_id')
    .withColumnRenamed('start_time', 'activity_start_time')
    .withColumnRenamed('devices_name', 'device_name')
    .withColumnRenamed('devices_id', 'device_id')
    .withColumnRenamed('tracks_start_time', 'track_start_time')
    .withColumnRenamed('tracks_artist', 'track_artist')
    .withColumnRenamed('tracks_album', 'track_album')
    .withColumnRenamed('tracks_title', 'track_title')
    .withColumnRenamed('tracks_player', 'track_player')
    .withColumnRenamed('tracks_id', 'track_id')
    .withColumnRenamed('tracks_uri', 'track_uri')
    
    # Place the columns belonging to the same group next to each other.
    .select('user_id',
            'activity_id', 'activity_start_time', 'activity_duration',
            'device_id', 'device_name',
            'track_artist', 'track_title', 'track_album',
            'track_player', 'track_start_time', 'track_duration',
            'track_id', 'track_uri')
    
    # Sort the data.
    .orderBy(f.asc('user_id'),
             f.asc('activity_start_time'))
)

# Print the execution time.
print(f'Execution time: {time.time() - start_time:.5f} s.')

# Display the top rows of the resulting table.
df_clean.limit(100).toPandas().head(5)

<hr style="border:2px solid black"></hr>

# Save the dataframe

In [ ]:
t.spark.save_dataframe_to_parquet(
    df_clean, Config.Path.project_data_root / 'df_1E6_clean.parquet')